In [1]:
conda env list

# conda environments:
#
base                     /home/juliaju_o/miniconda3
donut_official        *  /home/juliaju_o/miniconda3/envs/donut_official
jupyter_notebook         /home/juliaju_o/miniconda3/envs/jupyter_notebook
prokka1146               /home/juliaju_o/miniconda3/envs/prokka1146
quast                    /home/juliaju_o/miniconda3/envs/quast
singularity              /home/juliaju_o/miniconda3/envs/singularity
spades                   /home/juliaju_o/miniconda3/envs/spades


Note: you may need to restart the kernel to use updated packages.


In [10]:
# Donut: Document Understanding Transformer
# https://github.com/clovaai/donut

# https://github.com/clovaai/donut/issues/206
!pip install donut-python
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio
!pip install donut-python

In [11]:
import argparse
import gradio as gr
import torch
from PIL import Image

from donut import DonutModel

In [12]:
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

In [13]:
torch.cuda.is_available()

True

In [14]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="docvqa")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-docvqa")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:  # rvlcdip, cord, ...
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path, ignore_mismatched_sizes=True)
print(torch.cuda.is_available())

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

Some weights of the model checkpoint at naver-clova-ix/donut-base-finetuned-docvqa were not used when initializing DonutModel: ['encoder.model.norm.weight', 'encoder.model.norm.bias']
- This IS expected if you are initializing DonutModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DonutModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


True
IMPORTANT: You are using gradio version 3.34.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
